## Clustering

In [38]:
from sklearn import cluster, datasets
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.image import grid_to_graph
from sklearn.cluster import AgglomerativeClustering

In [6]:
iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target

In [7]:
k_means = cluster.KMeans(n_clusters=3)
k_means.fit(X_iris)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [8]:
print(k_means.labels_[::10])

[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]


In [9]:
print(y_iris[::10])

[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]


### Vector Quantization

In [10]:
try:
    face = sp.face(gray=True)
except AttributeError:
    from scipy import misc
    face = misc.face(gray=True)

In [13]:
face.shape

(768, 1024)

In [14]:
X = face.reshape((-1, 1))

In [16]:
k_means = cluster.KMeans(n_clusters=5, n_init=1)
k_means.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [23]:
values = k_means.cluster_centers_.squeeze()
labels = k_means.labels_

In [24]:
face_compressed = np.choose(labels, values)
face_compressed.shape = face.shape

In [25]:
face_compressed.shape

(768, 1024)

In [28]:
len(face_compressed)

768

## Hierarchical agglomerative clustering

### Connectivity-constrained clustering

In [51]:
try:  # SciPy >= 0.16 have face in misc
    from scipy.misc import face
    face = face(gray=True)
except ImportError:
    face = sp.face(gray=True)
    
face = sp.misc.imresize(face, 0.10) / 255.
X = np.reshape(face, (-1, 1))

connectivity = grid_to_graph(*face.shape)

AttributeError: module 'scipy.misc' has no attribute 'imresize'